In [ ]:
using QuickPOMDPs
using POMDPTools
using Distributions
using LinearAlgebra
using Plots
using MCTS

In [ ]:
# Define all possible states and actions (with constraints)
# [progress, power, h2o, o2]
max_progress = 5
max_power = 100
max_h2o = 10
max_o2 = 10
p_mission_regression = 0.05
p_power_fail = 0.02
p_water_loss = 0.02
energy_per_day = 2 #energy production per day

indexed = LinearIndices((max_progress, max_power, max_h2o, max_o2))
states = 1:indexed[max_progress, max_power, max_h2o, max_o2]
actions = [CartesianIndex(1, -4, -2, 0), #make mission progress
           CartesianIndex(0, -2, 1, 0), #make water
           CartesianIndex(0, -1, -1, 1), #make oxygen
           CartesianIndex(0, -0, 0, 0)] #do nothing
initialindex = indexed[1, 1, 1, 1]
#make mission progress

initialstate = Deterministic(initialindex)

In [ ]:
CartesianIndices((max_progress, max_power, max_h2o, max_o2))[1]

In [ ]:
indexed[2, 2, 2, 2]

In [ ]:
# Determine if action is out of bounds
# Associated consequences are reflected in transition/reward functions
state_maxes = [max_progress, max_power, max_h2o, max_o2]
function outofbounds(state, action)
    for i in 1:length(state)
        if ((state[i] + action[i]) < 1 || (state[i] + action[i]) > state_maxes[i])
            return true
        end
    end
    return false
end

In [ ]:
# Transition function
function transition(s, a)
    svec = CartesianIndices((max_progress, max_power, max_h2o, max_o2))[s]
    svec = collect(Tuple(svec))
    a = collect(Tuple(a))
    if outofbounds(svec, a)
        svec[2] = svec[2] + energy_per_day # increment energy
        if outofbounds(svec, a)
            svec[2] = max_power
        end
        spvec = svec
    else
        svec[2] = svec[2] + energy_per_day # increment energy
        if outofbounds(svec, a)
            svec[2] = max_power
        end
        x = rand()
        if (x < 0.05)
            spvec = svec # no progress
        elseif (x < p_water_loss)
            svec[3] = 0.0 # lose all water
            spvec = svec + a
        elseif (x<(p_water_loss + p_power_fail))
            svec[2] = 0.0 # lose all energy
            spvec = svec + a
        elseif (x < (p_mission_regression + p_water_loss + p_power_fail) && svec[3] >= 1)
            svec[1] = svec[1] - 1 # backwards progress mission
            spvec = svec + a
        else 
            spvec = svec + a
        end
    end
    spvec = CartesianIndices(spvec)
    sp = indexed[spvec]
end


In [ ]:
mission = QuickMDP(
    states = states,
    actions = actions,
    initialstate = initialstate,
    discount = 0.95,
    isterminal = s -> s[1] == 10,

    transition = transition,

    reward = function (s, a)
        svec = CartesianIndices((max_progress, max_power, max_h2o, max_o2))[s]
        spvec = svec + a
        if svec[1] == max_progress
            r = 1000
        elseif outofbounds(svec, a) # If invalid action
            r = -1e9
        else
            r = -1
        end
        return r
    end
)


In [ ]:
solver = MCTSSolver(n_iterations=50, depth=40, exploration_constant=5.0)
planner = solve(solver, mission)
#a = action(planner, s)

In [ ]:
#prepare to plot

#change and iterate like if (states == energy plot xy)
states = reshape(states, (4, Int(length(states)/4)))
mission = states[1,:]
resources_E = states[2, :]
resources_H2O = states[3,:]
resources_total = resources_E  + resources_H2O
goal = fill(mission[end], length(mission))
t = range(0, length=length(mission))
#rewards_accumulation = cumsum(rewards)

In [39]:
#plot Mission Progress over Resources
p1 = plot()
plot!(p1, mission, resources_total, marker=(:circle,5), arrow=true, arrowsize=0.1, legend=false)
vline!(p1, goal[1])
ylabel!("Resources")
xlabel!("Mission Progress")
ylims!(0.9, (state_dimensions[2] + state_dimensions[3] + 1))
xlims!(0.9, state_dimensions[1] + 0.5)

#savefig(p1, "Mission Progress VS Resources.pdf")

In [ ]:
#plot Mission and Resource development over time
#savefig(p2, "Mission Progress VS Resources.pdf")
anim = @animate for i in 1:length(mission)
    p2 = plot()
    plot!(p2 , 1:i, goal[1:i],  linewidth=2,   label="Mission Goal")
    plot!(p2 ,1:i, [resources_E[1:i], resources_H2O[1:i]], label=["Resources Energy" "Resources H2O"], title = "Resource and Mission development over time")
    plot!(p2 ,1:i, mission[1:i], linewidth = 2,  label="Mission Progress")
    xlabel!("Time Steps")
    ylabel!("Mission Progress and Resources")
end


gif(anim, fps=5)

In [ ]:
#Resource level histogram
p3 = bar(t, [resources_total, resources_E], xlabel="Time Steps", ylabel="Resource Accumulation", label=["Resources H20" "Resources Energy"], title = "Resource Reserves over time")

#savefig(p3, "Resource Reserves.pdf")

In [ ]:
#Plot Reward accumulation over time
layout = @layout [a b] #creates 1x2 subplots

p4 = plot(t[1:end-1], rewards[1:end-1], subplot=1, layout=layout,legend=false)
plot!(p4, t, rewards_accumulation, subplot=2, legend=false)
xlabel!(p4, "Time Steps")
ylabel!(p4, "Rewards", subplot=1)
ylabel!(p4, "Accumulated Rewards", subplot=2)
title!(p4,"Rewards (except the \n last time step)", subplot = 1)
title!(p4,"Accumulated Rewards", subplot = 2)

#savefig(p4, "Rewards.pdf")